<a href="https://colab.research.google.com/github/IkramKheopsys/airflow-spark-docker/blob/main/duckdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install duckdb==0.6.1
!pip install pandas==1.5.2
!pip install seaborn==0.12.1
!pip install matplotlib==3.6.2





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 3.4 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.10.3
    Uninstalling duckdb-0.10.3:
      Successfully uninstalled duckdb-0.10.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
malloy 2023.1067 requires duckdb<1.0.0,>=0.8.0, but you have duckdb 0.6.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.2 which is 

In [ ]:
import pandas as pd
import glob
import time
import duckdb

conn = duckdb.connect()
# with pandas
cur_time = time.time()
#df = pd.concat([pd.read_csv(f) for f in glob.glob('dataset/*.csv')])
df = pd.read_csv('/content/Sales_Product_Combined.csv')
print(f"time: {(time.time() - cur_time)}")
print(df.head(10))

time: 0.19460463523864746
   Order ID                     Product  Quantity Ordered     Price  \
0    176558        USB-C Charging Cable                 2     11.95   
1    176559  Bose SoundSport Headphones                 1     99.99   
2    176560                Google Phone                 1       600   
3    176560            Wired Headphones                 1     11.99   
4    176561            Wired Headphones                 1     11.99   
5    176562        USB-C Charging Cable                 1     11.95   
6    176563  Bose SoundSport Headphones                 1     99.99   
7    176564        USB-C Charging Cable                 1     11.95   
8    176565          Macbook Pro Laptop                 1  1,700.00   
9    176566            Wired Headphones                 1     11.99   

   Order Date      Time                        Purchase Address  \
0  19-04-2019   8:46 AM            917 1st St, Dallas, TX 75001   
1  07-04-2019  10:30 PM       682 Chestnut St, Boston, MA 

In [ ]:
# with duckdb
cur_time = time.time()
df = conn.execute("""
	SELECT *
	FROM read_csv_auto('/content/Sales_Product_Combined.csv',header=True)
	LIMIT 10
""").df()
print(f"time: {(time.time() - cur_time)}")
df

time: 0.04109358787536621


,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,8:46 AM,"917 1st St, Dallas, TX 75001",Dallas,Cable
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,10:30 PM,"682 Chestnut St, Boston, MA 02215",Boston,Headphones
2,176560,Google Phone,1,600,2019-04-12,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Phone
3,176560,Wired Headphones,1,11.99,2019-04-12,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Headphones
4,176561,Wired Headphones,1,11.99,2019-04-30,9:27 AM,"333 8th St, Los Angeles, CA 90001",Los Angeles,Headphones
5,176562,USB-C Charging Cable,1,11.95,2019-04-29,1:03 PM,"381 Wilson St, San Francisco, CA 94016",San Francisco,Cable
6,176563,Bose SoundSport Headphones,1,99.99,2019-04-02,7:46 AM,"668 Center St, Seattle, WA 98101",Seattle,Headphones
7,176564,USB-C Charging Cable,1,11.95,2019-04-12,10:58 AM,"790 Ridge St, Atlanta, GA 30301",Atlanta,Cable
8,176565,Macbook Pro Laptop,1,"1,700.00",2019-04-24,10:38 AM,"915 Willow St, San Francisco, CA 94016",San Francisco,Laptop
9,176566,Wired Headphones,1,11.99,2019-04-08,2:05 PM,"83 7th St, Boston, MA 02215",Boston,Headphones


In [ ]:
df = conn.execute("""
	SELECT *
	FROM '/content/Sales_Product_Combined.csv'
""").df()
conn.register("df_view", df) #store it as virtual table
conn.execute("DESCRIBE df_view").df() # doesn't work if you don't register df as a virtual table

,column_name,column_type,null,key,default,extra
0,Order ID,INTEGER,YES,NaN,NaN,NaN
1,Product,VARCHAR,YES,NaN,NaN,NaN
2,Quantity Ordered,INTEGER,YES,NaN,NaN,NaN
3,Price,VARCHAR,YES,NaN,NaN,NaN
4,Order Date,TIMESTAMP,YES,NaN,NaN,NaN
5,Time,VARCHAR,YES,NaN,NaN,NaN
6,Purchase Address,VARCHAR,YES,NaN,NaN,NaN
7,City,VARCHAR,YES,NaN,NaN,NaN
8,Product Type,VARCHAR,YES,NaN,NaN,NaN


In [ ]:
conn.execute("SELECT COUNT(*) FROM df_view").df()

,count_star()
0,185950


In [ ]:
df.isnull().sum()
df = df.dropna(how='all')

# Notice we use df and not df_view
# With DuckDB you can run SQL queries on top of Pandas dataframes
conn.execute("SELECT COUNT(*) FROM df").df()



conn.execute("DESCRIBE df_view").df()

,column_name,column_type,null,key,default,extra
0,Order ID,INTEGER,YES,NaN,NaN,NaN
1,Product,VARCHAR,YES,NaN,NaN,NaN
2,Quantity Ordered,INTEGER,YES,NaN,NaN,NaN
3,Price,VARCHAR,YES,NaN,NaN,NaN
4,Order Date,TIMESTAMP,YES,NaN,NaN,NaN
5,Time,VARCHAR,YES,NaN,NaN,NaN
6,Purchase Address,VARCHAR,YES,NaN,NaN,NaN
7,City,VARCHAR,YES,NaN,NaN,NaN
8,Product Type,VARCHAR,YES,NaN,NaN,NaN


In [ ]:
conn.execute("""
CREATE OR REPLACE TABLE sales AS
	SELECT
		"Order ID"::INTEGER AS order_id,
		Product AS product,
		"Quantity Ordered"::INTEGER AS quantity,
		"PRICE"::DECIMAL AS price_each,
		strptime("Order Date", '%m/%d/%Y %H:%M')::DATE as order_date,
		"Purchase Address" AS purchase_address
	FROM df
	WHERE
		TRY_CAST("Order ID" AS INTEGER) NOTNULL
""")

ConversionException: Conversion Error: Could not convert string "1,700.00" to DECIMAL(18,3)

In [ ]:
conn.execute("""
	CREATE OR REPLACE VIEW aggregated_sales AS
	SELECT
		order_id,
		COUNT(1) as nb_orders,
		MONTH(order_date) as month,
		str_split(purchase_address, ',')[2] AS city,
		SUM(quantity * PRICE) AS revenue
	FROM sales
	GROUP BY ALL
""")

CatalogException: Catalog Error: Table with name sales does not exist!
Did you mean "pg_am"?